<center><h1>PROYECTO ANALISIS DE ALGORITMOS <br> Javier Felipe Vasquez Roldan</h1></center>

<h2>MST Grafos de Aristas Rojas y Azules</h2>

In [9]:
class Node:
    def __init__(self, distance, color):
        self.d = distance
        self.c = color
        
class Parent:
    def __init__(self, parent, color):
        self.p = parent
        self.c = color

In [10]:
import numpy as np
import math
import sys


inf = math.inf

f = open("input_n8_b5_r2","r") 


for i,line in enumerate(f):
    if i==0:
        n_nodes= int(line)
        nodes = [[Node(inf,"N") for j in range(n_nodes)] for i in range(n_nodes)] 
    elif i==1:
        blue_arists = int(line)
    else:
        for j,part in enumerate(line.split()):
            if part!="Inf":
                nodes[i-2][j+i-2]=Node(int(part.split("/")[0]),part.split("/")[1])
                nodes[j+i-2][i-2]=Node(int(part.split("/")[0]),part.split("/")[1])


V = n_nodes
parent = [Node(0,"N") for i in range(V)]
key = [sys.maxsize for i in range(V)]
mstSet = [0 for i in range(V)]

    
def minKey(key,mstSet):
    min = sys.maxsize
 
    for v in range (V):
        if mstSet[v] == 0 and key[v] < min:
            min = key[v]
            min_index = v
    return min_index;

 
def imprimir(parent, n, graph):
    sum = 0 
    for i in range (1,V):
        sum = sum + graph[i][parent[i].p].d
    print (sum)
    for i in range (1,V):
        if parent[i].c=="B":
            print ( "(",parent[i].p,",", i,")",end='')
    print("")
    for i in range (1,V):
        if parent[i].c=="R":
            print ( "(",parent[i].p,",", i,")",end='')
        #,graph[i][parent[i].p].d,"/",parent[i].c

def primMST(graph,max_blues,faltantes):
    V = n_nodes
    parent = [Node(0,"N") for i in range(V)]
    key = [sys.maxsize for i in range(V)]
    mstSet = [0 for i in range(V)]
    key[0] = 0
    parent[0] = Parent(-1,"N")
    blues = 0

    for count in range (V):
        u = minKey(key, mstSet)
        mstSet[u] = 1
        for v in range (V):
            if mstSet[v] == 0 and graph[u][v].d <  key[v]:
                if faltantes == 0:
                    parent[v]  = Parent(u,graph[u][v].c)
                    key[v] = graph[u][v].d
                else:
                    if graph[u][v].c == "B":
                        parent[v]  = Parent(u,graph[u][v].c)
                        key[v] = graph[u][v].d
                        faltantes=faltantes-1
                
    for i in range (1,V):
        if parent[i].c=="B":
            blues = blues+1
    if blues<max_blues:
        faltantes = max_blues-blues
        primMST(graph,max_blues,faltantes)
    else:
        imprimir(parent, V, graph)




In [12]:
primMST(nodes,blue_arists,0)

22
( 4 , 2 )( 1 , 3 )( 0 , 4 )( 2 , 6 )( 6 , 7 )
( 2 , 1 )( 1 , 5 )

# Solucion 2

La solucion 2 se realiza apartir de una busqueda de todos los posibles casos de un MST del grafo hasta que se llega a uno que cumpla con los requisitos minimos de aristas rojas y azules, en resumidas cuentas se implementa un algoritmo de fuerza bruba el cual:

- Genera todos los grafos posibles con todos los nodos
- Analiza si este tiene ciclos (si no, entonces es un MST)
- Revisa si cumple con el numero de aristas rojas definida por el archivo
- Retorna la solucion

Esta misma solucion se tiene una complejidad de $O(2ˆn)$

In [21]:
import numpy as np
import math
import networkx as nx
import sys

inf = math.inf


def leer_archivo(file):
    f = open(file,"r") 
    for i,line in enumerate(f):
        if i==0:
            n_nodes= int(line)
            nodes = [[Node(inf,"#") for j in range(n_nodes)] for i in range(n_nodes)] 
        elif i==1:
            blue_arists = int(line)
        else:
            for j,part in enumerate(line.split()):
                if part!="Inf":
                    nodes[i-2][j+i-2]=Node(int(part.split("/")[0]),part.split("/")[1])
                    nodes[j+i-2][i-2]=Node(int(part.split("/")[0]),part.split("/")[1])
                    
    return n_nodes, blue_arists, nodes

def imprimir(parent, n, graph):
    sum = 0 
    for i in range (1,V):
        sum = sum + graph[i][parent[i].p].d
    print (sum)
    for i in range (1,V):
        if parent[i].c=="B":
            print ( "(",parent[i].p,",", i,")",end='')
    print("")
    for i in range (1,V):
        if parent[i].c=="R":
            print ( "(",parent[i].p,",", i,")",end='')
        #,graph[i][parent[i].p].d,"/",parent[i].c

In [13]:
# CUENTA ARISTAS AZULES EN EL GRAFO
def azules(A):
    cont = 0;
    for i in A:
        for j in i:
            if(j.c == 'B'):
                cont += 1
    return cont
# CUENTA ARISTAS ROJAS EN EL GRAFO
def rojos(A):
    return len(A) - numBlue(A)

# CONVIERTE LA MATRIZ DE ADYACENCIA EN UNA MATRIZ SENCILLA SIN COLORES 
def matriz(A):
    T = []
    for i in range(len(A)):
        P = []
        for j in range(len(A)):
            P.append(0)
        T.append(P)
    for i in range (len(A)):
        for j in range (len(A)):
            if A[i][j].d != inf:
                T[i][j] = A[i][j].d
    return T

# CONVIERTE LA MATRIZ DE ADYACENCIA EN UNA MATRIZ NUMPHY Y REVISA SI TIENE CICLOS CON NETWORKX.CYCLE_BASIS
def ciclos(A):
    T = matriz(A)
    A=np.matrix(T)
    G=nx.from_numpy_matrix(A)
    C = nx.cycle_basis(G,0)
    if(len(C) == 0):
        return False
    else:
        return True    


n,K,V = leer_archivo('input_n8_b5_r2')


ans = []
minW = inf


def MST2(at, x, y, Sol):
    
#     for i in range (len(Sol)):
#         for j in range (len(Sol)):
#             print(Sol[i][j].c,end = " ")
#         print("")
#     print("------------------")

# REVISA SI EL GRAFO NO TIENE CICLOS Y SI CUMPLE CON EL NUMERO DE ARISTAS ROJAS Y AZULES 
    if(azules(Sol) == K and rojos(Sol) == n - K - 1 and ciclos(Sol, n) == False):
        ans = Sol
        return
    if(at == len(V)):
        return
    
    temp = Sol
#     SE QUITA UNA ARISTA PARA VER SI EXISTE UNA SOLUCION CON EL NUEVO GRAFO
    temp[x][y] = Node(inf,"#")
    temp[y][x] = Node(inf,"#")
    
#     print (x,y)

# SE ITERA POR TODOS LOS NODOS 
    MST2(at+1, x+1 , y, temp)
    MST2(at+1, x , y+1, temp)


Sol = MST2(0,0,0,V)
imprimir(sol)
for i in range (len(ans)):
        for j in range (len(ans)):
            print(ans[i][j].d)


22
( 4 , 2 )( 1 , 3 )( 0 , 4 )( 2 , 6 )( 6 , 7 )
( 2 , 1 )( 1 , 5 )

<h2>K- Tiendas</h2>

In [6]:
import numpy as np
import math
import sys


inf = math.inf

f = open("input_n5_k2.txt","r") 


for i,line in enumerate(f):
    if i==0:
        k= int(line)
    else:
        if i==1:
            size = len(line.split())-1
            matrix = [[inf for j in range(size)] for i in range(size)]
            names = ["x" for j in range(size)]
        for j,part in enumerate(line.split()):
            if j==0:
                names[i-1] = part
            else:
                if part!="0":
                    matrix[i-1][j-1]=int(part)
                
print (matrix)
print (names)



for i in range (size):
    for j in range (size):
        if()
        cost = cost + matrix[i][j]
               


SyntaxError: invalid syntax (<ipython-input-6-8867dd531f54>, line 33)

<h1> Cliques en Grafos </h1>

<p>Se usa una libreria llamada Networkx la cual maneja la creacion de los grafos y encuentra los cliques. Se implementa un algoritmo de Hamming que toma un tiempo de O(2^n) la implementacion es tomada de: https://gist.github.com/mariaperez/1896793</p>

In [20]:
import networkx
import itertools
import logging
import matplotlib.pyplot as plt

def hamming(sf, ss):        
        dif = 0
        for first, second in zip(sf, ss):
                if first != second:
                        dif += 1
        return dif
    
def hamming_binary(tam,d):
    space = networkx.Graph()

    l = ["01"] * tam 
    all_nodes = itertools.product(*l)
    all_nodes = [''.join(x) for x in all_nodes]
    logging.debug(all_nodes)
    space.add_nodes_from(all_nodes) 
    
    for node in space.nodes():        
        for v in space.nodes():
            if(v != node):
                dist = hamming(node, v)
                if(dist >= d):
                    space.add_edge(node,v)
    
    return space


def mutate_node(node, n):
    if node[n] == '0':
        newbase = '1'
    else:
        newbase = '0'
    new_node = node[0:n] + newbase + node[n+1:]
    return new_node

space = hamming_binary(4,3)

cliques =list(networkx.find_cliques(space) )
num_cliques =  networkx.graph_clique_number(space)

print (cliques)
print ("Numero de Cliques en el grafo: ",num_cliques)


[['1111', '1000'], ['1111', '0100'], ['1111', '0001'], ['1111', '0000'], ['1111', '0010'], ['0110', '1000'], ['0110', '1011'], ['0110', '0001'], ['0110', '1101'], ['0110', '1001'], ['0111', '1000'], ['0111', '1010'], ['0111', '1100'], ['0111', '0000'], ['0111', '1001'], ['1010', '1101'], ['1010', '0100'], ['1010', '0001'], ['1010', '0101'], ['1011', '1100'], ['1011', '0100'], ['1011', '0000'], ['1011', '0101'], ['0011', '1000'], ['0011', '0100'], ['0011', '1100'], ['0011', '1110'], ['0011', '1101'], ['1100', '0010'], ['1100', '0001'], ['1101', '0000'], ['1101', '0010'], ['1110', '0101'], ['1110', '0000'], ['1110', '0001'], ['1110', '1001'], ['0101', '1000'], ['0101', '0010'], ['1001', '0010'], ['1001', '0100']]
Numero de Cliques en el grafo:  2
